To detect movements in a video, we use background subtraction. There are many different background subtraction algorithms, and each algorithm has various parameters. You can use the default parameter settings in Birdwatcher, but you can also modify the parameters and optimize these for your own videos.

This notebook can be used to play around with all options and various parameters settings of background subtraction algorithms when using Birdwatcher for movement detection, and see how this influences the results of location detection.

In [ ]:
import birdwatcher as bw
import birdwatcher.movementdetection as md
from birdwatcher.plotting import imshow_frame # birdwatcher has vizualization tools

import matplotlib.pyplot as plt
%matplotlib inline

### Select video fragment

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zebrafinch.MTS')

Choose a short representative video fragment where the object of interest is moving quite a lot.

In [ ]:
startat = '00:02:10'   # in HOURS:MM:SS
duration = 20          # seconds

In [ ]:
# look at the chosen video fragment
vfs.iter_frames(startat=startat, nframes=vfs.avgframerate*duration).show(framerate=150)

### Choose parameters

In this example, we will use background subtraction MOG2, and try to find optimal parameter values for this algorithm. Also some manipulations before or after performing background subtraction might improve location detection, and therefore, you can also compare the settings of those.

First, decide which settings you would like, by adding various values in the list after each parameter. 

In [ ]:
settings = {'History': [4, 12],
            'ComplexityReductionThreshold': [0.05],
            'BackgroundRatio': [0.1, 0.5],
            'NMixtures': [7, 50],
            'VarInit': [15],
            'VarMin': [4],
            'VarMax': [75],
            'VarThreshold': [10, 70],
            'VarThresholdGen': [9],
            'DetectShadows': [False],
            'ShadowThreshold': [0.5],
            'ShadowValue': [127],

            'color': [True, False],   # booleans only
            'resizebyfactor': [1, (2/3)],   # use '1' for no change in size
            'blur': [0, 10],   # use '0' for no blur
            'morphologyex': [True, False]}   # booleans only

all_combinations = list(bw.product_dict(**settings))
print(f'There are {len(all_combinations)} different combinations of settings to perform movement detection.')

The higher the number of combinations, the longer the next step (running movement detection) will take. Another option is to start by tweaking some parameters with larger steps of parameter values, and fine-tune the values in next rounds.

### Run movemement detection per combination of settings

Movement detection is done for each combination of settings, and the mean coordinate per frame is saved in a Pandas dataframe.

**WARNING:** This step might take a while, depending on the number of settings combinations!

In [ ]:
%%time
params = md.apply_all_parameters(vfs, settings, bgs_type=bw.BackgroundSubtractorMOG2, 
                                 startat=startat, duration=duration)
params.df

Here, you see a pandas dataframe with in the columns all parameters that are used to run movement detection. The rows show the specific value of each parameter and the resulted mean x,y coordinates per frame (NaN means there were no nonzero pixels found for that frame). 

In [ ]:
params.get_info()

### Save ParameterSelection

Save the results as csv file, to be able to load the dataframe later without running all movement detections per setting combination again!

In [ ]:
# save DataFrame
params.save_parameters(f'output/')
params.path

The results are automatically saved in a folder with the name of the videofilestream. Often, several rounds of parameter selection per videofragment will be done with different parameter settings. Then, the same foldername is used with a number added as suffix to display the round.

If you want to choose your own foldername to save the results, you could add the desired foldername as optional argument. Also, you could add overwrite=False to replace an existing ParameterSelection project.

### Load ParameterSelection

In [ ]:
# load ParameterSelection project
params = md.load_parameterselection(f'output\params_zebrafinch')

Also, the associated videofilestream can be loaded:

In [ ]:
params.vfs

Or watch the videofragment of which the ParameterSelection object is based on:

In [ ]:
frames_fragment = params.get_videofragment()
frames_fragment.show()

To access the data, run:

In [ ]:
params.df

### Correction resizebyfactor

Setting 'resizebyfactor' changes the width and height of the frames. Below, we correct for this change in pixel resolution, so that it's easier to see and compare the effects of different settings on the movementdetection results below.

In [ ]:
params.df['pixel'] = params.df['pixel'] / params.df['resizebyfactor']
params.df.loc[:, ('resizebyfactor', 'coords', 'pixel')]

### Visualize results

Before visualizing the results, look again at all settings.

In [ ]:
# the following settings have been used for backgroundsubstraction in this dataframe
params.get_parameters('all')

Here, you see for which settings multiple values have been used to run movement detection. So, these are also the settings that are interesting to compare in plots or superimpose on the video fragment.

In [ ]:
# the following settings have been tested with multiple values
params.get_parameters('multi_only')

#### Plots

First, choose for each parameter with multiple values which value is the default. TIP: you can copy the output dictionary above and choose one of the values in each list. Use the value of which you think will provide the best location detection. If you have no idea, don't worry, just choose one.

In [ ]:
default_values = {'morphologyex': False,
                  'NMixtures': 7,
                  'blur': 10,
                  'BackgroundRatio': 0.1,
                  'color': False,
                  'History': 4,
                  'VarThreshold': 70,
                  'resizebyfactor': 1.0}

You can plot the results of two parameters in one figure. The different values of one parameter is outlined in the rows and the other parameter in the columns of the subplots.

In [ ]:
rows = 'History'
cols = 'color'

g = params.plot_parameters(rows, cols, default_values)

To save the plots of all combinations of parameters, use the function below.

In [ ]:
params.batch_plot_parameters(default_values)

The figures are saved in the same directory as where the associated ParameterSelection dataframe is saved. You can go to the folder where the figures are saved and walk through the figures. That way you get a sense of the influence of various parameter-value combinations on location detection.

For certain parameters, you might see large noise differences for the different values. For these parameters, choose the best value (the one with the least noise), and use these values as default. Run the above cells again with the new default values. The figures will be saved in a new folder (figures_2). Look again at the figures. Do this several rounds, untill you get an idea of which parameter-value combinations provide the best (least noisy) location detection.

#### Superimpose on video

In the plots you get an idea of which paramater-value combinations result in the least noisy graphs. However, it is not possible to see whether the pixel coordinates also accurately match the location of the bird. For this, it is usefull to plot the mean coordinates directly on top of the video.

In [ ]:
params.get_parameters('multi_only')

Again, look at the parameters with multiple values. Choose from these parameters which values you would like to see plotted as circle on the videofragment.

In [ ]:
# choose which settings to superimpose on the videofragment
settings = {'morphologyex': [True],
            'NMixtures': [7],
            'blur': [10],
            'BackgroundRatio': [0.1,],
            'color': [False, True],
            'History': [4],
            'VarThreshold': [10, 70],
            'resizebyfactor': [1.0]}

all_combinations = list(bw.product_dict(**settings))
print(f'There are {len(all_combinations)} combinations of settings to superimpose on a video.')

Too many circles plotted on the video are hard to follow. As default, a maximum of 6 circles can be superimposed on one videofragment, but often you'll probably want to plot less circles.

In [ ]:
# draw circles on videofragment
frames, colorspecs = params.draw_multiple_circles(settings)

In [ ]:
# show the settings for each color of the circles
colorspecs

In [ ]:
# look at the video using show()
frames.show(framerate=20)

TIP: a lower framerate makes it easier to follow the circles.

In [ ]:
# or, save as video with circles superimposed
vfs_circles = frames.tovideo(f'{params.path}/multicircles.mp4', framerate=params.vfs.avgframerate)

In [ ]:
# you can also save the color specification
colorspecs.to_csv(f'{params.path}/multicircles_colorspecs.csv')

Now, you have an idea which parameters have a large influence on movement detection. You might want to run the notebook again and test some more values for these parameters to fine-tune your results even more. Just repeat all the steps above.

Also, repeat these steps with a second short representative videofragment to make sure the same parameter-value combinations provide the best results. After that, you could use these settings to run movement detect on all your videos.